# **Load the Data and Import Libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [19]:
import numpy as np
import torch
from torch import nn, optim
from torch.utils import data
from torchsummary import summary
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import math
import cv2

import os
import gc
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
data_path = "/content/gdrive/MyDrive/Asteroid RL dataset/new_RL_preset/data_pole_axis_RL_preset_batch_0.npy"
data_RL_preset0 = np.load(data_path)

In [ ]:
data_path1 = "/content/gdrive/MyDrive/Asteroid RL dataset/new_RL_preset/data_pole_axis_RL_preset_batch_1.npy"
data_RL_preset1 = np.load(data_path1)

data_RL_preset0[0, 0] = data_RL_preset0[0, 0] + data_RL_preset1[0, 0]
data_RL_preset0 = np.concatenate((data_RL_preset0, data_RL_preset1[1:, :]), axis=0)
del data_RL_preset1
gc.collect()

In [22]:
data_path2 = "/content/gdrive/MyDrive/Asteroid RL dataset/new_RL_preset/data_pole_axis_RL_preset_batch_2.npy"
data_RL_preset2 = np.load(data_path2)

In [ ]:
print(data_RL_preset0[0, 0])
print(data_RL_preset0[0, 1])
print(data_RL_preset0[0, 2])

gc.collect()

In [7]:
class RewardMapModifier():
    def __init__(self, extends=(0, 1), blur_coef=(5, 3)):
        self.extends = extends
        self.blur_coef = blur_coef

    def extend_hori(self, reward_map, action_maps):
        left_reward = reward_map[..., :, -int(reward_map.shape[-2]*self.extends[1]/2):, :]
        right_reward = reward_map[..., :, :int(reward_map.shape[-2]*self.extends[1]/2), :]

        if action_maps is not None:
            left_actions = action_maps[..., :, -int(action_maps.shape[-2]*self.extends[1]/2):, :].copy()
            right_actions = action_maps[..., :, :int(action_maps.shape[-2]*self.extends[1]/2), :].copy()
            left_actions[..., :, :, 0] = left_actions[..., :, :, 0] - 1
            right_actions[..., :, :, 0] = right_actions[..., :, :, 0] + 1

        if self.extends[1] != 0:
            extended_reward = np.concatenate((left_reward, reward_map, right_reward), axis=-2)
            extended_actions = np.concatenate((left_actions, action_maps, right_actions), axis=-2) if action_maps is not None else action_maps
        else:
            extended_reward = reward_map
            extended_actions = action_maps

        return extended_reward, extended_actions

    def extend_vert(self, reward_map, action_maps):
        top_reward = np.flip(reward_map[..., 1:int(reward_map.shape[-3]*self.extends[0]/2), :, :], -3)
        bottom_reward = np.flip(reward_map[..., -int(reward_map.shape[-3]*self.extends[0]/2):-1, :, :], -3)

        if action_maps is not None:
            top_actions = np.flip(action_maps[..., 1:int(action_maps.shape[-3]*self.extends[0]/2), :, :].copy(), -3)
            bottom_actions = np.flip(action_maps[..., -int(action_maps.shape[-3]*self.extends[0]/2):-1, :, :].copy(), -3)
            top_actions[..., :, :, 1] = 2*0 - top_actions[..., :, :, 1]
            bottom_actions[..., :, :, 1] = 2*1 - bottom_actions[..., :, :, 1]

        if self.extends[0] != 0:
            extended_reward = np.concatenate((top_reward, reward_map, bottom_reward), axis=-3)
            extended_actions = np.concatenate((top_actions, action_maps, bottom_actions), axis=-3) if action_maps is not None else action_maps
        else:
            extended_reward = reward_map
            extended_actions = action_maps

        return extended_reward, extended_actions

    def blur(self, reward_map):
        #reward_map = 2.5 * np.tan( reward_map * (np.pi/2) / 6 )
        if len(reward_map.shape) == 3:
            reward_map[:, :, 0] = cv2.GaussianBlur(reward_map[:, :, 0], (self.blur_coef[0], self.blur_coef[0]), self.blur_coef[1])
        elif len(reward_map.shape) == 4:
            for i in range(reward_map.shape[0]):
                reward_map[i, :, :, 0] = cv2.GaussianBlur(reward_map[i, :, :, 0], (self.blur_coef[0], self.blur_coef[0]), self.blur_coef[1])
        reward_map = 6 * (2/np.pi) * np.arctan(reward_map/2)
        return reward_map

    def operation(self, reward_map, action_maps, order=['extend_hori', 'extend_vert', 'blur']):
        result_reward = reward_map
        result_action = action_maps
        for op in order:
            if op == 'extend_hori':
                result_reward, result_action = self.extend_hori(result_reward, result_action)
            elif op == 'extend_vert':
                result_reward, result_action = self.extend_vert(result_reward, result_action)
            elif op == 'blur':
                result_reward = self.blur(result_reward)
            else:
                raise NotImplementedError()
        return result_reward, result_action

    def ext_N_set(self, N_set):
        return (N_set[0]+2*int(N_set[0]*self.extends[1]/2), N_set[1]+2*int(N_set[1]*self.extends[0]/2))


class EarlyStopping():
    def __init__(self, patience, delta, mode='min'):
        """
        patience : max number of waiting
        delta : min boundary of "change"
        mode :
        verbose :
        """

        self.patience = patience
        self.delta = delta
        self.mode = mode
        self.best_score = np.inf if mode == 'min' else 0
        self.count = 0
        self.early_stop = False

    def __call__(self, score):
        if self.mode == 'min':
            if (self.best_score - score) < self.delta:
                self.count += 1
            else:
                self.best_score = score
                self.count = 0
        elif self.mode == 'max':
            if (score - self.best_score) < self.delta:
                self.count += 1
            else:
                self.best_score = score
                self.count = 0

        if self.count >= self.patience:
            self.early_stop = True

def data_split(dataset, train_ratio=0.7, shuffle=True, copy=False):
    if shuffle:
        idx = np.arange(0, dataset.shape[0])
        np.random.shuffle(idx)
        dataset = dataset[idx]

    trainset = dataset[:int(train_ratio*dataset.shape[0])]
    testset = dataset[int(train_ratio*dataset.shape[0]):]
    if copy:
        trainset = trainset.copy()
        testset = testset.copy()

    return trainset, testset

# **Training with Regression Model**

In [8]:
class QValueNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, activation=nn.ReLU, dropout=0.3):
        super().__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.activation = activation

        self.model = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            activation(),
            nn.Dropout(dropout),

            nn.Linear(self.hidden_dim, self.hidden_dim),
            activation(),
            nn.Dropout(dropout),

            #------------------------------
            nn.Linear(self.hidden_dim, self.hidden_dim),
            activation(),
            nn.Dropout(dropout),

            #nn.Linear(self.hidden_dim, self.hidden_dim),
            #activation(),
            #nn.Dropout(dropout),

            nn.Linear(self.hidden_dim, self.hidden_dim//4),
            activation(),
            nn.Dropout(dropout),

            nn.Linear(self.hidden_dim//4, self.hidden_dim//8),
            activation(),
            nn.Dropout(dropout),

            nn.Linear(self.hidden_dim//8, 1)
        )

    def forward(self, X):
        return self.model(X)
"""

class QValueNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, activation=nn.ReLU, dropout=0.3):
        super().__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.activation = activation

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(3, 3), stride=(1, 1)), #padding 2
            activation(),
            nn.Dropout(dropout),
            nn.MaxPool2d(kernel_size=2, stride=2)
        ) # (1X10X20 -> 1X14X24 (padding)) -> 4X5X10

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), #padding 2
            activation(),
            nn.Dropout(dropout),
            nn.MaxPool2d(kernel_size=2, stride=2)
        ) # 4X5X10 -> 16X2X5

        self.linear_model = nn.Sequential(
            nn.Linear(459, 512),
            activation(),
            nn.Dropout(dropout),

            nn.Linear(512, 512),
            activation(),
            nn.Dropout(dropout),

            nn.Linear(512, 512),
            activation(),
            nn.Dropout(dropout),

            nn.Linear(512, 1),
            activation(),
            nn.Dropout(dropout),
        )

    def padding_cir(self, X:torch.Tensor, pad=(2, 2)):
        X_new = X.clone()
        X_new = torch.cat((X[..., :, -pad[0]:], X_new, X[..., :, :pad[0]]), axis=-1)
        X_new = torch.cat((torch.flip(X_new[..., :pad[1], :], [-2]), X_new, torch.flip(X_new[..., -pad[1]:, :], [-2])), axis=-2)

        return X_new

    def forward(self, X):
        X_r_arr = X[..., :200]
        if len(X.shape) == 1:
            batch_size = 1
        else:
            batch_size = X.shape[0]
        X_r_arr = (X_r_arr.reshape((batch_size, -1, 20, 10))).transpose(-2, -1)/torch.mean(X_r_arr)
        X_r_arr_padded = self.padding_cir(X_r_arr, pad=(1, 1))
        X_info = X[..., 200:]

        x = self.conv1(X_r_arr_padded)
        x = self.conv2(x)
        x = torch.flatten(x, start_dim=1)
        x = torch.cat((x, torch.squeeze(X, dim=1)), dim=1)
        x = self.linear_model(x)

        return x
"""
class CustomLoss(nn.Module):
    def __init__(self, relative, percent):
      super().__init__()
      self.relative = relative
      self.percent = percent

    def forward(self, input, target):
      torch_MSE = nn.MSELoss()
      if self.relative:
          loss = torch_MSE(input/(target+1e-6), target/(target+1e-6))
          loss = torch.sqrt(loss + 1e-6)
      else:
          loss = torch.sqrt(torch_MSE(input, target))
          #weight = 0.5 + 0.5*torch.abs(target)
          #loss = torch.sqrt(torch.sum(weight*(input-target)**2)/torch.sum(weight) + 1e-6)
      if self.percent:
          loss = 100 * loss
      return loss

def train_loop(dataloader, model, loss_fn, optimizer, train_loss, es:EarlyStopping):
    epoch_loss = 0
    n_train = 0

    model.train()
    #with torch.autograd.detect_anomaly(True):
    for X_train, y_train in dataloader:
        X_train = X_train.to(device)
        y_train = y_train.to(device)
        pred = model(X_train)

        non_extended = torch.logical_and((X_train[:, -4] >= 0), (X_train[:, -4] < 1))
        non_extended = torch.logical_and(non_extended, (X_train[:, -3] >= 0))
        non_extended = torch.logical_and(non_extended, (X_train[:, -3] < 1))
        loss = loss_fn(pred[non_extended], y_train[non_extended])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()*X_train.size(0)
        n_train += X_train.size(0)

    epoch_loss /= n_train
    train_loss.append(epoch_loss)

    es(epoch_loss)
    #print("train_loss : {:9.4g}".format(epoch_loss), end=' ')

def test_loop(dataloader, model, loss_fn, test_loss, epoch):
    epoch_loss = 0
    n_test = 0

    model.eval()
    with torch.no_grad():
        for X_test, y_test in dataloader:
            X_test = X_test.to(device)
            y_test = y_test.to(device)
            pred = model(X_test)

            non_extended = torch.logical_and((X_test[:, -4] >= 0), (X_test[:, -4] < 1))
            non_extended = torch.logical_and(non_extended, (X_test[:, -3] >= 0))
            non_extended = torch.logical_and(non_extended, (X_test[:, -3] < 1))
            epoch_loss += loss_fn(pred[non_extended], y_test[non_extended]).item()*X_test.size(0)
            n_test += X_test.size(0)

    epoch_loss /= n_test
    test_loss.append(epoch_loss)

    print("train_loss : {:9.4g}".format(train_loss[-1]), end=' ')
    print("| test_loss : {:9.4g}".format(epoch_loss), end=' ')
    print("\n", end=' ')

# Data Processing : scaling data
param = [6, 2] #[6, 2.5]
def scale_reward(data):
    if data_RL_preset0[0, 2] == 1: # already scaled
        return data

    data_RL_preset0[0, 2] = 1
    scaled_data = np.zeros_like(data)

    scaled_data = param[0]*(2/np.pi)*np.arctan(data/param[1])

    return scaled_data

def test_img_show(i_img):
    fig = plt.figure(figsize=(16, 8), dpi=300)
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)

    extent = ( (N_set[0]-ext_N_set[0])/2, (N_set[0]+ext_N_set[0])/2, (N_set[1]-ext_N_set[1])/2, (N_set[1]+ext_N_set[1])/2 )
    if i_img == 0 or True:
        ax1.clear()
        im1 = ax1.imshow(test_img_list[i_img], vmin=-param[0], vmax=param[0], extent=extent)
        ax1.set_title("TEST_IMAGE_"+str(i_img))
        plt.colorbar(im1, ax=ax1, fraction=0.026, pad=0.04)
        ax1.plot([0, N_set[0]],        [0, 0],               color='red', linestyle='solid')
        ax1.plot([0, N_set[0]],        [N_set[1], N_set[1]], color='red', linestyle='solid')
        ax1.plot([0, 0],               [0, N_set[1]],        color='red', linestyle='solid')
        ax1.plot([N_set[0], N_set[0]], [0, N_set[1]],        color='red', linestyle='solid')

    reward_map_temp = np.zeros((resol*N_set[0], resol*N_set[1]))
    model.eval()
    with torch.no_grad():
        for idx in range(N_set[0]*N_set[1]*resol*resol):
            i = idx//int(resol*N_set[1])
            j = idx%int(resol*N_set[1])
            phi_action = (i/(resol*N_set[0]))%1
            theta_action = (j/(resol*N_set[1]))%1

            state = test_img_data[i_img*resol*N_set[0]*N_set[1], :906]
            actions = np.array([phi_action, theta_action, 0.1, 0.1])

            input = torch.tensor(np.concatenate((state, actions))).float().to(device)
            reward = model(input)
            reward_map_temp[i, j] = reward
    ax2.clear()
    im2 = ax2.imshow(reward_map_temp.T)#, vmin=-param[0], vmax=param[0])
    ax2.set_title("MODEL_OUTPUT_"+str(i_img))
    plt.colorbar(im2, ax=ax2, fraction=0.026, pad=0.04)

    plt.show()

### Data Preprocessing with Small Size Dataset

In [9]:
class Dataset(data.Dataset):
    def __init__(self, x_tensor, y_tensor):
        super(Dataset, self).__init__()

        if not torch.is_tensor(x_tensor):
            self.x = torch.tensor(x_tensor).float()
            self.y = torch.tensor(y_tensor).float()
        else:
            self.x = x_tensor.float()
            self.y = y_tensor.float()

    def __getitem__(self, index): return self.x[index], self.y[index]

    def __len__(self): return self.x.shape[0]

In [ ]:
# seed
seed = 722
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# hyperparameters
batch_size = 1024
learning_rate = 6e-5
max_epoch = 1000

# other parameters
N_set = (40, 20)
resol = 1

map_modifier = RewardMapModifier(extends=(0, 0), blur_coef=(3, 2))
chunk_size = 256
chunk_set_size = chunk_size*N_set[0]*N_set[1]
online_dataset_path = "/content/gdrive/MyDrive/Asteroid RL dataset/online_dataset/"


data_len2 = int(data_RL_preset2[0, 0])
test_img_num = 10
test_img_idx_choice = np.random.randint(0, (data_len2-1)//800, 10)
dataset_img_idx = np.full(data_len2, False)
for i in test_img_idx_choice:
    dataset_img_idx[i*800+1:(i+1)*800+1] = True

data_RL_preset = data_RL_preset0[1:, :]
test_img_data = data_RL_preset2[dataset_img_idx, :].copy()
del data_RL_preset2
gc.collect()

test_img_list = []
for i in range(test_img_num):
    test_img_list.append(test_img_data[i*resol*N_set[0]*N_set[1]:(i+1)*resol*N_set[0]*N_set[1], -1].reshape((N_set[0], N_set[1])).T)

for i in range(len(test_img_list)):
    test_img_list[i], _ = map_modifier.operation(np.expand_dims(test_img_list[i], axis=-1), None, order=['extend_hori', 'extend_vert', 'blur'])
    #test_img_list[i] = test_img_list[i][:, :, 0]
    gc.collect()


cut = N_set[0]*N_set[1]*2093 + 1 #1040
state_data = data_RL_preset[:cut, :-5]
action_data = data_RL_preset[:cut, -5:-1]
reward_data = data_RL_preset[:cut, -1:]


new_action_data = 0 * np.array([action_data[0, ...].copy()])
new_reward_data = 0 * np.array([reward_data[0, ...].copy()])

print("Data Shapes Before Map Modifying")
print("--------------------------------")
print("state_data  | "+str(state_data.shape)+", "+str(int(1000*state_data.itemsize*state_data.size/(2**30))/1000)+"GB")
print("action_data | "+str(action_data.shape)+"  , "+str(int(1000*action_data.itemsize*action_data.size/(2**30))/1000)+"GB")
print("reward_data | "+str(reward_data.shape)+"  , "+str(int(1000*reward_data.itemsize*reward_data.size/(2**30))/1000)+"GB")

print("\n--------------------------------")
for i in range(math.ceil(state_data.shape[0]/chunk_set_size)):
    if i != state_data.shape[0]//(chunk_size*N_set[0]*N_set[1]):
        reward_map = reward_data[chunk_set_size*i:chunk_set_size*(i+1)]
        action_maps = action_data[chunk_set_size*i:chunk_set_size*(i+1)]
    else:
        reward_map = reward_data[chunk_set_size*i:]
        action_maps = action_data[chunk_set_size*i:]

    print("Batch Shape : reward / action | "+str(reward_map.shape)+", "+str(action_maps.shape)+" --> ", end='')
    reward_map = np.swapaxes(reward_map.reshape((-1, N_set[0], N_set[1], 1)), -2, -3)
    action_maps = np.swapaxes(action_maps.reshape((-1, N_set[0], N_set[1], 4)), -2, -3)
    reward_map, action_maps = map_modifier.operation(reward_map, action_maps, order=['extend_hori', 'extend_vert', 'blur'])
    print(str(reward_map.shape)+", "+str(action_maps.shape))

    extended_size = reward_map.shape[-2] * reward_map.shape[-3]
    new_action_data = np.concatenate((new_action_data, action_maps.reshape(-1, 4)), axis=0)
    new_reward_data = np.concatenate((new_reward_data, reward_map.reshape(-1, 1)), axis=0)
print("--------------------------------\n")

state_data = np.repeat(state_data[::N_set[0]*N_set[1]], repeats=extended_size, axis=0)
action_data = np.delete(new_action_data, 0, axis=0)
reward_data = np.delete(new_reward_data, 0, axis=0)

del new_action_data, new_reward_data, reward_map, action_maps
del data_RL_preset, data_RL_preset0
gc.collect()

print("Data Shapes After Map Mpdifying")
print("--------------------------------")
print("state_data  | "+str(state_data.shape)+", "+str(int(1000*state_data.itemsize*state_data.size/(2**30))/1000)+"GB")
print("action_data | "+str(action_data.shape)+"  , "+str(int(1000*action_data.itemsize*action_data.size/(2**30))/1000)+"GB")
print("reward_data | "+str(reward_data.shape)+"  , "+str(int(1000*reward_data.itemsize*reward_data.size/(2**30))/1000)+"GB")

ext_N_set = map_modifier.ext_N_set(N_set)


total_data = np.concatenate((state_data, action_data, reward_data), axis=1)
state_shape = state_data.shape[1]
del state_data, action_data, reward_data
gc.collect()

train_data, test_data = data_split(total_data, train_ratio=0.85, shuffle=True, copy=True)
del total_data
gc.collect()

train_state_data = train_data[:, :state_shape].copy()
train_action_data = train_data[:, state_shape:state_shape+4].copy()
train_reward_data = train_data[:, -1].reshape(-1, 1)
del train_data
gc.collect()

test_state_data = test_data[:, :state_shape].copy()
test_action_data = test_data[:, state_shape:state_shape+4].copy()
test_reward_data = test_data[:, -1].reshape(-1, 1)
del test_data
gc.collect()

train_dataset = Dataset(np.concatenate((train_state_data, train_action_data), axis=1), train_reward_data)
test_dataset = Dataset(np.concatenate((test_state_data, test_action_data), axis=1), test_reward_data)

train_dataloader = data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

del train_state_data, train_action_data, train_reward_data
del test_state_data, test_action_data, test_reward_data
gc.collect()

## **Training Part**

In [ ]:
# hyperparameters
learning_rate = 8e-5
max_epoch = 400
print(torch.__file__)

model = QValueNet(input_dim=910, hidden_dim=1024, activation=nn.ELU, dropout=0.15).to(device)
summary(model, (1, model.input_dim))

optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)
loss_fn = CustomLoss(relative=False, percent=False)

train_loss = []
test_loss = []

es = EarlyStopping(patience=2000, delta=0.1)
for epoch in tqdm(range(max_epoch)):
    #print("EPOCH "+str(epoch)+" TRAINING...")
    train_loop(train_dataloader, model, loss_fn, optimizer, train_loss, es)
    #print("EPOCH "+str(epoch)+" TESTING...")
    test_loop(test_dataloader, model, loss_fn, test_loss, epoch)
    #print("")

    if es.early_stop:
        print("EarlyStop Triggered : Bestscore = {:7.4g}".format(es.best_score))
        break

    if (epoch+1)%10 == 0 and epoch != 0:
        plt.figure(figsize=(5, 3), dpi=300)
        plt.plot(train_loss[2:], label='train_loss')
        plt.plot(test_loss[2:], label='test_loss')
        plt.legend()
        plt.title("Train/Test Loss (MSE)")
        plt.show()

        for i in range(test_img_num):
            #if (i > 3 and i < 15) or i > 19:
            #    continue
            test_img_show(i)

    print("[epochs:{:2}]".format(epoch+2), end='')

print("DONE")

plt.figure(dpi=300)
plt.plot(train_loss[2:], label='train_loss')
plt.plot(test_loss[2:], label='test_loss')
plt.legend()
plt.title("Train/Test Loss (MSE)")
plt.show()

In [ ]:
for i in range(test_img_num):
    test_img_show(i)